In [1]:
# Import core libraries
import pandas as pd
import numpy as np
import datetime
# Set random seed for reproducibility
np.random.seed(42)

### Helper Function 


In [2]:
#Helper functions
def gen_id(data):
    """Create a unique identifier from group, number, generation.

    Args:
        data (pd.DataFrame): raw data
    Returns:
        pd.DataFrame: data with an additional 'id' column
    """
    data['id'] = data['Group'].astype(str) + '_' + data['NR'].astype(str) + '_' + data['Generation'].astype(str)
    return data

def make_rat_one_row(data):
    """From 4 trials converts each rat into a single row.

    Args:
        data (pd.DataFrame): raw data
    Returns:
        pd.DataFrame: data with each rat as a single row
    """
    re_df = pd.DataFrame()
    for id in data['id'].unique():
        temp = data[data['id'] == id]
        to_merge = temp.loc[temp['Trials'] == 1]
        for i in range(2, 5):
            tmp = temp.loc[temp['Trials'] == i].drop(columns=params['no_merge_cols'])
            to_merge = pd.merge(to_merge, tmp, how='inner', on="id", suffixes=('', f'_{i}'))

        re_df = pd.concat([re_df, to_merge], ignore_index=True)
    return re_df
        

### Configuration Parameters
This dictionary defines:
- The path to the raw dataset
- Columns to drop during the cleaning process
- String-based columns that should be standardized to lowercase

These parameters make the cleaning functions reusable and adaptable to other datasets.

In [3]:
params = {
    #Original data path
    'data' : '../raw/ambitus_0_15_log_29_07_2025.parquet',
    #Columns from the original dataset that are not needed for the analysis 
    'to_drop_cols' : ['Separation', 'Date_Ambitus', 'GR_Sex', 'G_S', 'Animal', 'EAT_E_Nr', 'EAT_I_Nr', 'EAT_TOT_Nr', 'Expl_E_BEF_Nr', 'Expl_I_BEF_Nr', 'Expl_E_I_BEF_Nr', 'Expl_E_AFT_Nr', 'Expl_I_AFT_Nr', 
                      'Expl_E_I_AFT_Nr', 'Expl_E_BEF_T', 'Expl_I_BEF_T', 'Expl_E_I_BEF_T', 'Expl_E_AFT_T', 'Expl_I_AFT_T', 'Expl_E_I_AFT_T', 'LAT_E', 'LAT_I', 'LAT_E_I', 'Expl_REP_E_BEF_Nr', 'Expl_REP_I_BEF_Nr',
                      'Expl_REP_BEF_Nr', 'Expl_REP_E_AFT_Nr', 'Expl_REP_I_AFT_Nr', 'Expl_REP_AFT_Nr', 'LOCO_BEF', 'L_C_Tot',
                      'Expl_E_AFT_T_Calc','Expl_I_AFT_T_Calc','Expl_E_I_AFT_T_Calc','Expl_E_BEF_Calc_T','Expl_I_BEF_Calc_T','Expl_E_I_BEF_Calc_T','Expl_E_AFT_Calc','Expl_I_AFT_Calc','Expl_E_I_AFT_Calc',
                      'Expl_REP_E_AFT_Nr_Calc',	'Expl_REP_I_AFT_Nr_Calc',	'Expl_REP_AFT_Nr_Calc'],
    'no_merge_cols': ['Group', 'NR', 'Generation', 'Trials', 'Paradigm'],
    'save_date':'29_07_2025'
}

### Load and Annotate the Raw Dataset
We load the raw data from the specified parquet file and apply the gen_id() function to add a unique identifier (id) for each animal. This step prepares the data for downstream cleaning and ensures consistent identification of records.

In [4]:
parquet_data = pd.read_parquet(params['data'])
parquet_data = gen_id(parquet_data)
parquet_data

,Animal,Generation,Season,Separation,G_S,Paradigm,Date_Ambitus,Year,NR,Group,...,Eff_Expl_E,Eff_Expl_I,Eff_Expl_EI,E_E,WM_ERROR_E,WM_ERROR_I,WM_ERROR_EI,WM_ERR_RW,WM_ERR_NRW,id
0,LE2F1_1,0,Autumn,2019-08-26,0_Autumn,1,2019-10-07 00:00:00,0,1,Lisket,...,1.000,0.75,0.875000,0.8750,0.040000,0.059406,0.088235,0.088235,NaN,Lisket_1_0
1,LE2F2_1,0,Autumn,2019-08-26,0_Autumn,1,2019-10-07 00:00:00,0,1,Lisket,...,1.000,1.00,1.000000,1.0000,0.034653,0.051471,0.082353,0.082353,NaN,Lisket_1_0
2,LE2F3_1,0,Autumn,2019-08-26,0_Autumn,2,2019-10-07 00:00:00,0,1,Lisket,...,NaN,1.00,0.533333,1.0000,0.066176,0.105882,0.030000,0.105882,0.030000,Lisket_1_0
3,LE2F4_1,0,Autumn,2019-08-26,0_Autumn,2,2019-10-07 00:00:00,0,1,Lisket,...,NaN,1.00,0.500000,1.0000,0.105882,0.030000,0.035433,0.030000,0.035433,Lisket_1_0
4,LE2F1_2,0,Autumn,2019-08-26,0_Autumn,1,2019-10-07 00:00:00,0,2,Lisket,...,1.000,1.00,1.000000,1.0000,0.033333,0.039370,0.094340,0.094340,NaN,Lisket_2_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,Rat28,15,Winter,2024-12-30,15_Winter,2,2025-02-18 00:00:00,6,98,LE,...,NaN,1.00,0.533333,1.0000,0.096386,0.026667,0.040201,0.026667,0.040201,LE_98_15
5364,Rat11,15,Winter,2024-12-30,15_Winter,1,2025-02-18 00:00:00,6,99,LE,...,0.875,1.00,0.937500,0.9375,0.030000,0.045226,0.116883,0.116883,NaN,LE_99_15
5365,Rat12,15,Winter,2024-12-30,15_Winter,1,2025-02-18 00:00:00,6,99,LE,...,1.000,1.00,1.000000,1.0000,0.070352,0.181818,0.120690,0.120690,NaN,LE_99_15
5366,Rat33,15,Winter,2024-12-30,15_Winter,2,2025-02-18 00:00:00,6,99,LE,...,NaN,1.00,0.533333,1.0000,0.103896,0.068966,NaN,0.068966,NaN,LE_99_15


## Reduce and Restructure Dataset
To prepare the dataset for machine learning:
- We remove unused metadata columns.
- We restructure the data so that each rat is represented by a single row using make_rat_one_row().
- (Optionally) We standardize feature names or values (e.g., converting % to "perc").

This format is suitable for classical machine learning algorithms that expect tabular inputs.

In [7]:
# Remove unnecessary columns based on the parameter list
parquet_data = parquet_data.drop(columns=params['to_drop_cols'])
# Transform the dataset so that each rat is represented by a single row
one_rowed_data = make_rat_one_row(parquet_data)
# Optionally standardize column names or values
one_rowed_data = one_rowed_data.replace('male', 'Male')
one_rowed_data = one_rowed_data.replace('female', 'Female')
one_rowed_data = one_rowed_data.fillna(-1)
one_rowed_data

,Generation,Season,Paradigm,Year,NR,Group,Sex,Trials,EAT_E_%,EAT_I_%,...,Expl_E_I_TOT_Loco_ratio_4,Eff_Expl_E_4,Eff_Expl_I_4,Eff_Expl_EI_4,E_E_4,WM_ERROR_E_4,WM_ERROR_I_4,WM_ERROR_EI_4,WM_ERR_RW_4,WM_ERR_NRW_4
0,0,Autumn,1,0,1,Lisket,Male,1,100.0,75.0,...,4.214286,-1.0,1.0,0.500000,1.0,0.105882,0.030000,0.035433,0.030000,0.035433
1,0,Autumn,1,0,2,Lisket,Male,1,87.5,75.0,...,3.647059,-1.0,1.0,0.571429,1.0,0.072072,0.026667,0.036036,0.026667,0.036036
2,0,Autumn,1,0,3,Lisket,Male,1,100.0,75.0,...,5.153846,-1.0,1.0,0.571429,1.0,0.065217,0.020000,0.020000,0.020000,0.020000
3,0,Autumn,1,0,4,LE,Male,1,87.5,87.5,...,6.545455,-1.0,1.0,0.571429,1.0,0.076923,0.030000,0.030000,0.030000,0.030000
4,0,Autumn,1,0,5,Lisket,Male,1,50.0,75.0,...,5.111111,-1.0,1.0,0.533333,1.0,0.055556,0.023333,0.023333,0.023333,0.023333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,15,Winter,1,6,95,LE,Male,1,87.5,50.0,...,2.684211,-1.0,1.0,0.500000,1.0,0.083333,0.036667,0.036667,0.036667,0.036667
1338,15,Winter,1,6,96,LE,Male,1,100.0,75.0,...,2.826087,-1.0,1.0,0.571429,1.0,0.145455,0.026667,0.049383,0.026667,0.049383
1339,15,Winter,1,6,97,LE,Male,1,100.0,75.0,...,2.800000,-1.0,1.0,0.666667,1.0,0.096774,0.025105,0.073171,0.025105,0.073171
1340,15,Winter,1,6,98,LE,Male,1,100.0,100.0,...,2.312500,-1.0,1.0,0.533333,1.0,0.096386,0.026667,0.040201,0.026667,0.040201


### Drop Redundant or Non-Feature Columns
We automatically remove columns that are not needed for machine learning, including:
- Redundant group/Sex/year fields (duplicated across time)
- Intermediate or repeated trial-level metadata
- Columns with placeholder names like Unnamed:

This keeps only relevant, flat (one-row-per-animal) behavioral features in the dataset.

In [8]:
#Drop columns that are not needed for the analysis
columns_to_drop = one_rowed_data.filter(regex='Group_|Sex_|Season_|Trials_|NR_|Year_|Paradigm_|Generation_|Unnamed|First_Box|Skipping').columns
one_rowed_data = one_rowed_data.drop(columns=columns_to_drop)
#Drop columns that are only contains -1 values
one_rowed_data = one_rowed_data.loc[:, (one_rowed_data != -1).any(axis=0)]
one_rowed_data

,Generation,Season,Paradigm,Year,NR,Group,Sex,Trials,EAT_E_%,EAT_I_%,...,Expl_I_TOT_Loco_ratio_4,Expl_E_I_TOT_Loco_ratio_4,Eff_Expl_I_4,Eff_Expl_EI_4,E_E_4,WM_ERROR_E_4,WM_ERROR_I_4,WM_ERROR_EI_4,WM_ERR_RW_4,WM_ERR_NRW_4
0,0,Autumn,1,0,1,Lisket,Male,1,100.0,75.0,...,2.357143,4.214286,1.0,0.500000,1.0,0.105882,0.030000,0.035433,0.030000,0.035433
1,0,Autumn,1,0,2,Lisket,Male,1,87.5,75.0,...,2.117647,3.647059,1.0,0.571429,1.0,0.072072,0.026667,0.036036,0.026667,0.036036
2,0,Autumn,1,0,3,Lisket,Male,1,100.0,75.0,...,3.076923,5.153846,1.0,0.571429,1.0,0.065217,0.020000,0.020000,0.020000,0.020000
3,0,Autumn,1,0,4,LE,Male,1,87.5,87.5,...,3.727273,6.545455,1.0,0.571429,1.0,0.076923,0.030000,0.030000,0.030000,0.030000
4,0,Autumn,1,0,5,Lisket,Male,1,50.0,75.0,...,3.222222,5.111111,1.0,0.533333,1.0,0.055556,0.023333,0.023333,0.023333,0.023333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,15,Winter,1,6,95,LE,Male,1,87.5,50.0,...,1.578947,2.684211,1.0,0.500000,1.0,0.083333,0.036667,0.036667,0.036667,0.036667
1338,15,Winter,1,6,96,LE,Male,1,100.0,75.0,...,1.565217,2.826087,1.0,0.571429,1.0,0.145455,0.026667,0.049383,0.026667,0.049383
1339,15,Winter,1,6,97,LE,Male,1,100.0,75.0,...,1.450000,2.800000,1.0,0.666667,1.0,0.096774,0.025105,0.073171,0.025105,0.073171
1340,15,Winter,1,6,98,LE,Male,1,100.0,100.0,...,1.343750,2.312500,1.0,0.533333,1.0,0.096386,0.026667,0.040201,0.026667,0.040201


Save the Cleaned Dataset
After removing the final unused column (Trials), we export the cleaned and restructured dataset to the processed/ directory.
The filename includes the current date, allowing for versioning and traceability.

Resulting file example:
 - ambitus_0_15_ml_ready_29-07-2025.csv

In [9]:
# Drop the 'Trials' column if it's no longer relevant
one_rowed_data.drop(columns=['Trials'], inplace=True)
# Save the cleaned and flattened dataset to CSV with timestamp
date = params['save_date']
one_rowed_data.to_csv(f"../processed/ambitus_0_15_ml_ready_{date}.csv", index=False)